<h1>Automated Gating with Immunova: Loading peritonitis data into database</h1>

This is the first of three notebooks comparing the automated gating in immunova to expert manual gating. In this notebook I am creating the database entries.

<h3>Create project and experiments</h3>

In [12]:
from immunova.data.project import Project
from immunova.data.mongo_setup import global_init
from immunova.data.fcs_experiments import FCSExperiment, Panel
from immunova.data.utilities import get_fcs_file_paths
from immunova.flow.readwrite.read_fcs import explore_channel_mappings, FCSFile
from tqdm import tqdm_notebook, tqdm
from functools import partial
import pandas as pd
global_init()
root_drive = '/media/rossco/FCS_DATA'

In [2]:
pd_project = Project(project_id='Peritonitis', owner='burtonrossj')
pd_project.save()

<Project: Project object>

In [3]:
pd_project.id.__str__()

'5d84fca4975ee6db94526c28'

I will create four experiments in total:
* PBMC_T: T cell panel for PBMC samples
* PBMC_M: Myeloid cell panel for PBMC samples
* PDMC_T: T cell panel for peritoneal fluid samples
* PDMC_M: Myeloid cell panel for peritoneal fluid samples

For each of these experiments I need to associate a flow cytometry panel. A Panel object defines the channel(fluorochrome)/marker(antibody) mappings for all associated flow data. This allows for standardisation of the flow cytometry meta-data at the point of entry.

Panel objects can be created from a python dictionary object or using an excel template. In this case I have created an excel template (see documentation for details on creating panel templates).

It is often useful to explore the channel and marker names of a large selection of fcs files to get a feel for the naming conventions and make sure you have convered all edge cases. There is a useful utility function in `immunova.flow.readwrite.read_fcs` called `explore_channel_mappings`. Given a directory, the function will search for all `.fcs` files and return all permutations of channel/marker pairings found.

In [ ]:
cm_permutations = explore_channel_mappings('/media/ross/FCS_DATA/Raya PD Samples/ds_friendly')

In [6]:
len(cm_permutations)

20

So there is 20 permutations for the different ways that markers have been labelled in fcs files. I can account for most cases using regular expression but in a few cases (e.g. live/dead staining) I have added like-for-like matches in the templates.

In [4]:
t_panel = Panel(panel_name='peritonitis_t_panel')
m_panel = Panel(panel_name='peritonitis_m_panel')

In [5]:
t_panel.create_from_excel(path='experiment_data/peritonitis_t_template.xlsx')

True

In [6]:
m_panel.create_from_excel(path='experiment_data/peritonitis_m_template.xlsx')

True

The `create_from_excel` method will populate the Panel object using the excel template. I can now save the panels to the database.

In [7]:
t_panel.save()
m_panel.save()

<Panel: Panel object>

With the panels created I can now create the experiments. When you create an experiment you must always associate it to a project. We therefore use the `add_experiment` method of the Project object.

In [8]:
pbmc_t = pd_project.add_experiment(experiment_id='PBMC_T', panel_name='peritonitis_t_panel')
pdmc_t = pd_project.add_experiment(experiment_id='PDMC_T', panel_name='peritonitis_t_panel')
pbmc_m = pd_project.add_experiment(experiment_id='PBMC_M', panel_name='peritonitis_m_panel')
pdmc_m = pd_project.add_experiment(experiment_id='PDMC_M', panel_name='peritonitis_m_panel')

Experiment created successfully!
Experiment created successfully!
Experiment created successfully!
Experiment created successfully!


Now that the experiments are created I can start adding the fcs files. The `add_new_sample` method is used to generate a new fcs file entry into the mongo database, which is then associated to the experiment. See the documentation below:


In [9]:
?pbmc_t.add_new_sample

Signature:
pbmc_t.add_new_sample(
    sample_id: str,
    file_path: str,
    controls: list,
    comp_matrix: <built-in function array> = None,
    compensate: bool = True,
    feedback: bool = True,
) -> str
Docstring:
Add a new sample (FileGroup) to this experiment
:param sample_id: primary ID for identification of sample (FileGroup.primary_id)
:param file_path: file path of the primary fcs file (e.g. the fcs file that is of primary interest such as the
file with complete staining)
:param controls: list of file paths for control files e.g. a list of file paths for associated FMO controls
:param comp_matrix: (optional) numpy array for spillover matrix for compensation calculation; if not supplied
the matrix linked within the fcs file will be used, if not present will present an error
:param compensate: boolean value as to whether compensation should be applied before data entry (default=True)
:param feedback: boolean value, if True function will provide feedback in the form of print 

<h3>Add PDMC files (Myeloid cell panel)</h3>

A summary table of all the samples collected in the peritonitis study can provide us with the sample numbers and the manual gating results.

In [13]:
summary = pd.read_excel(f'{root_drive}/Raya PD Samples/ClinicalData_and_ManualGatingResults.xlsx')

In [14]:
pdmc_sample_ids = summary[summary['Cell origin'] == 'PDMC']['Patient no.'].values

In [15]:
pdmc_sample_ids

array(['142-09', '175-09', '209-03', '209-05', '210-12', '210-14',
       '229-02', '237-06', '239-02', '239-04', '251-07', '251-08',
       '254-04', '254-05', '255-04', '255-05', '262-01', '264-02',
       '267-01', '267-02', '272-01', '273-01', '276-01', '279-03',
       '286-02', '286-03', '286-04', '288-02', '289-01', '294-01',
       '294-02', '294-03', '295-01', '298-01', '302-01', '305-01',
       '305-02', '305-03', '306-01', '307-01', '308-01', '308-02R',
       '308-03R', '308-04', '310-01', '315-01', '315-02', '316-01',
       '318-01', '320-01', '321-01', '322-01', '323-01', '323-02',
       '324-01', '326-01'], dtype=object)

We can use the utility functin `get_fcs_file_paths` from immunova's data module to generate file paths for adding samples.

In [18]:
get_fcs_file_paths(fcs_dir=f'{root_drive}/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel',
                  control_names=['CD1c', 'HLA-DR'], ctrl_id='FMO')

{'primary': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N1_013.fcs'],
 'controls': [{'control_id': 'CD1c',
   'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N2 FMO CD1c_014.fcs'},
  {'control_id': 'HLA-DR',
   'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N3 FMO HLA-DR_015.fcs'}]}

In [19]:
pdmc_m_142_09 = get_fcs_file_paths(fcs_dir=f'{root_drive}/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel',
                  control_names=['CD1c', 'HLA-DR'], ctrl_id='FMO')
primary = pdmc_m_142_09['primary'][0]
controls = pdmc_m_142_09['controls']

In [20]:
pdmc_m.add_new_sample(sample_id='pd142_09_m', file_path=primary, controls=controls)

Generating main file entry...
Generating file entries for controls...
Successfully created pd142_09_m and associated to PDMC_M


'5d84fd30975ee6db9452715e'

Let's add the rest of the files for PDMC's.

In [21]:
#pd209-03 M
# Only one file for this patient
primary = f'{root_drive}/Raya PD Samples/ds_friendly/PDMC/209-03/m_panel/Peri 209-03 PDMC_A1_001.fcs'
pdmc_m.add_new_sample(sample_id='pd209-03_m', controls=[], file_path=primary)

Generating main file entry...
Generating file entries for controls...
Successfully created pd209-03_m and associated to PDMC_M


'5d84fd41975ee6db945274c0'

In [22]:
from flowio import FlowData
fcs = FlowData(primary)

In [23]:
fcs.channels

{'1': {'PnN': 'FSC-A'},
 '2': {'PnN': 'FSC-H'},
 '3': {'PnN': 'SSC-A'},
 '4': {'PnN': 'SSC-H'},
 '5': {'PnN': 'SSC-W'},
 '6': {'PnN': 'Alexa Fluor 488-A', 'PnS': 'CD14'},
 '7': {'PnN': 'Alexa Fluor 647-A', 'PnS': 'Siglec-8'},
 '8': {'PnN': 'APC-Cy7-A', 'PnS': 'CD3'},
 '9': {'PnN': 'Alexa Fluor 405-A', 'PnS': 'CD1c'},
 '10': {'PnN': 'AmCyan-A', 'PnS': 'Zombie Aqua'},
 '11': {'PnN': 'BV605-A', 'PnS': 'CD15'},
 '12': {'PnN': 'BV711-A', 'PnS': 'HLA-DR'},
 '13': {'PnN': 'PE-A', 'PnS': 'CD116'},
 '14': {'PnN': 'PE-Cy7-A', 'PnS': 'CD19'},
 '15': {'PnN': 'Alexa Fluor 700-A', 'PnS': 'CD45'},
 '16': {'PnN': 'Time'}}

pd209-03 for M panel is missing CD16

In [24]:
# pd209-05 M
# Only one file for this patient
primary = f'{root_drive}/Raya PD Samples/ds_friendly/PDMC/209-05/m_panel/Stable 209-05 PDMC N1 FSC 350_013.fcs'
pdmc_m.add_new_sample(sample_id='pd209-05_m', controls=[], file_path=primary)

Generating main file entry...
Generating file entries for controls...
Successfully created pd209-05_m and associated to PDMC_M


'5d84fd4c975ee6db945274d3'

In [25]:
# Add some convenience variables/partialised func
from functools import partial
root = f'{root_drive}/Raya PD Samples/ds_friendly/PDMC'
get_fp = partial(get_fcs_file_paths, control_names=['CD1c', 'HLA-DR'], ctrl_id='FMO')

In [26]:
# pd210-14 M
file_paths = get_fp(f'{root}/210-14/m_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [27]:
file_paths

{'primary': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/m_panel/Peri 210-14 PDMC N1_019.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/m_panel/Peri 210-14 PDMC N2_020.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/m_panel/Peri 210-14 PDMC N3_021.fcs'],
 'controls': []}

File names missing name of the FMO and therefore not captured by my function. I know that N2 is the FMO for CD1c and N3 is the FMO for HLA-DR

In [28]:
primary = file_paths['primary'][0]
controls = [dict(control_id='CD1c', path=file_paths['primary'][1]),
           dict(control_id='HLA-DR', path=file_paths['primary'][2])]

In [29]:
pdmc_m.add_new_sample(sample_id='pd210-14_m', controls=controls, file_path=primary)

Generating main file entry...
Generating file entries for controls...
Successfully created pd210-14_m and associated to PDMC_M


'5d84fd70975ee6db94527934'

In [30]:
# pd237-06 M
file_paths = get_fp(f'{root}/237-06/m_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [31]:
file_paths

{'primary': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/m_panel/Peri 237-06 PDMC N1_001.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/m_panel/Peri 237-06 PDMC N2_002.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/m_panel/Peri 237-06 PDMC N3_003.fcs'],
 'controls': []}

In [32]:
primary = file_paths['primary'][0]
controls = [dict(control_id='CD1c', path=file_paths['primary'][1]),
           dict(control_id='HLA-DR', path=file_paths['primary'][2])]

In [33]:
pdmc_m.add_new_sample(sample_id='pd237-06_m', controls=controls, file_path=primary)

Generating main file entry...
Generating file entries for controls...
Successfully created pd237-06_m and associated to PDMC_M


'5d84fd92975ee6db9452808e'

In [34]:
# pd239-02 M
file_paths = get_fp(f'{root}/239-02/m_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [35]:
file_paths

{'primary': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/239-02/m_panel/239-02 Stable PDMC Neat N1_014.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/239-02/m_panel/239-02 Stable PDMC Neat N2_015.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/239-02/m_panel/239-02 Stable PDMC Neat N3_016.fcs'],
 'controls': []}

In [36]:
primary = file_paths['primary'][0]
controls = [dict(control_id='CD1c', path=file_paths['primary'][1]),
           dict(control_id='HLA-DR', path=file_paths['primary'][2])]

In [37]:
pdmc_m.add_new_sample(sample_id='pd239-02_m', controls=controls, file_path=primary)

Generating main file entry...
Generating file entries for controls...
Successfully created pd239-02_m and associated to PDMC_M


'5d84fdab975ee6db9452871a'

In [38]:
# pd239-03
file_paths = get_fp(f'{root}/239-04/m_panel')

I now know the general errors that arrise now so I can loop over the remaining files and catch errors where necessary.

In [39]:
from immunova.data.fcs import FileGroup
processed_so_far = FileGroup.objects()
processed_so_far = [f.primary_id.replace('_m', '').replace('pd', '') for f in processed_so_far]
processed_so_far

['142_09', '209-03', '209-05', '210-14', '237-06', '239-02']

In [40]:
remaining = [p for p in pdmc_sample_ids if p not in processed_so_far]

In [41]:
# Add some convenience variables/partialised func
root = f'{root_drive}/Raya PD Samples/ds_friendly/PDMC'
get_fp = partial(get_fcs_file_paths, control_names=['CD1c', 'HLA-DR'], ctrl_id='FMO')
no_files = []
errors = []

for pt_id in tqdm_notebook(remaining):
    file_paths = get_fp(f'{root}/{pt_id}/m_panel')
    if not file_paths['controls']:
        if not file_paths['primary']:
            no_files.append(pt_id)
            continue
        if len(file_paths['primary']) == 1:
            print(f'Single file found for {pt_id}')
            primary = file_paths['primary'][0]
            controls = []
            pdmc_m.add_new_sample(sample_id=f'pd{pt_id}_m', 
                                  controls=controls, 
                                  file_path=primary)
        else:
            primary = []
            controls = []
            for f in file_paths['primary']:
                if f.find('N2') != -1:
                    controls.append(dict(control_id='CD1c', path=f))
                if f.find('N3') != -1:
                    controls.append(dict(control_id='HLA-DR', path=f))
                if f.find('N1') != -1:
                    primary.append(f)
            if len(primary) > 1:
                errors.append(pt_id)
                continue
            if len(controls) > 2:
                errors.append(pt_id)
                continue
            primary = primary[0]
            pdmc_m.add_new_sample(sample_id=f'pd{pt_id}_m', 
                      controls=controls, 
                      file_path=primary)
    else:
        if not file_paths['primary']:
            errors.append(pt_id)
            continue
        if len(file_paths['primary'])>1:
            errors.append(pt_id)
            continue
        if len(file_paths['controls'])>3:
            errors.append(pt_id)
            continue
        if len(file_paths['primary']) == 1 and len(file_paths['controls']) == 3:
            primary = file_paths['primary'][0]
            controls = file_paths['controls']
            pdmc_m.add_new_sample(sample_id=f'pd{pt_id}_m', 
                                  controls=controls, 
                                  file_path=primary)

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Single file found for 239-04
Generating main file entry...
Generating file entries for controls...
Successfully created pd239-04_m and associated to PDMC_M
Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Generating main file entry...
Generating file entries for controls...
Successfully created pd251-07_m and associated to PDMC_M
Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Generating main file entry...
Generating file entries for controls...
Successfully created pd251-08_m and associated to PDMC_M
Single file found for 254-04
Generating main file entry...
Generating file entries for controls...
Successfully created pd254-04_m and associated to PDMC_M
Single file found for 254-05
Generating main file entry...
Generating file entries for controls...
Successfully created pd254-05_m and associated to PDMC_M
Singl

In [42]:
no_files

['175-09',
 '229-02',
 '262-01',
 '289-01',
 '294-01',
 '308-04',
 '320-01',
 '321-01',
 '322-01',
 '323-02',
 '324-01',
 '326-01']

In [43]:
errors

['210-12']

210-12 has duplicate files. It has also been flagged as unusable in manual gating so I will ignore it.

<h3>Add PDMC files (T cell panel)</h3>

In [44]:
root = f'{root_drive}/Raya PD Samples/ds_friendly/PDMC'
get_fp = partial(get_fcs_file_paths, control_names=['CXCR3', 'CD27', 'CD45RA',
                                                   'CCR7'], ctrl_id='FMO')

In [45]:
pdmc_t = FCSExperiment.objects(experiment_id='PDMC_T').get()
no_files = []
errors = []

for pt_id in tqdm_notebook(pdmc_sample_ids):
    file_paths = get_fp(f'{root}/{pt_id}/t_panel')
    if not file_paths['controls']:
        if not file_paths['primary']:
            no_files.append(pt_id)
            continue
        if len(file_paths['primary']) == 1:
            print(f'Single file found for {pt_id}')
            primary = file_paths['primary'][0]
            controls = []
            pdmc_t.add_new_sample(sample_id=f'pd{pt_id}_t', 
                                  controls=controls, 
                                  file_path=primary)
        else:
            primary = []
            controls = []
            for f in file_paths['primary']:
                if f.find('T2') != -1:
                    controls.append(dict(control_id='CXCR3', path=f))
                if f.find('T3') != -1:
                    controls.append(dict(control_id='CD27', path=f))
                if f.find('T4') != -1:
                    controls.append(dict(control_id='CD45RA', path=f))
                if f.find('T5') != -1:
                    controls.append(dict(control_id='CCR7', path=f))
                if f.find('T1') != -1:
                    primary.append(f)
            if not primary:
                errors.append(pt_id)
                continue                
            if len(primary) > 1:
                errors.append(pt_id)
                continue
            if len(controls) > 2:
                errors.append(pt_id)
                continue
            primary = primary[0]
            pdmc_t.add_new_sample(sample_id=f'pd{pt_id}_t', 
                      controls=controls, 
                      file_path=primary)
    else:
        if not file_paths['primary']:
            errors.append(pt_id)
            continue
        if len(file_paths['primary'])>1:
            errors.append(pt_id)
            continue
        if len(file_paths['controls'])>4:
            errors.append(pt_id)
            continue
        if len(file_paths['primary']) == 1 and len(file_paths['controls']) == 4:
            primary = file_paths['primary'][0]
            controls = file_paths['controls']
            pdmc_t.add_new_sample(sample_id=f'pd{pt_id}_t', 
                                  controls=controls, 
                                  file_path=primary)

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Single file found for 175-09
Generating main file entry...
Generating file entries for controls...
Successfully created pd175-09_t and associated to PDMC_T
Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Single file found for 209-05
Generating main file entry...
Generating file entries for controls...
Successfully created pd209-05_t and associated to PDMC_T
Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Warning! Multiple non-control (primary) files found in directory. Check before proceeding.
Single file found for 239-02
Generating main file entry...
Generating file entries for controls...
Successfully created pd239-02_t and associated to PDMC_T
Single file found for 239-04
Generating main file entry...
Generating file en

In [46]:
errors

['142-09',
 '209-03',
 '210-12',
 '210-14',
 '237-06',
 '254-05',
 '264-02',
 '267-02',
 '273-01',
 '276-01',
 '286-03',
 '288-02',
 '295-01',
 '302-01',
 '305-02',
 '306-01',
 '308-03R',
 '315-01',
 '315-02',
 '316-01',
 '318-01',
 '320-01',
 '323-01']

Let's address the errors:

* 142-09 - No T cells, ignore
* 209-03 - No T cells, ignore
* 210-12 - No T cells, ignore
* 210-14 - unclear why this didn't load, will repeat
* 237-06 - unclear why this didn't load, will repeat
* 254-05 - files missing
* 264-02 - unclear why this didn't load, will repeat
* 267-02 - unclear why this didn't load, will repeat
* 273-01 - unclear why this didn't load, will repeat
* 276-01 - unclear why this didn't load, will repeat
* 286-03 - unclear why this didn't load, will repeat
* 288-02 - unclear why this didn't load, will repeat
* 295-01 - Flow experiment failed, ignore
* 302-01 - Done in duplicate; use 44-48
* 305-02 - unclear why this didn't load, will repeat
* 306-01 - Unstained file needs removing
* 308-03R - unclear why this didn't load, will repeat
* 315-01 - unclear why this didn't load, will repeat
* 315-02 - unclear why this didn't load, will repeat
* 316-01 - unclear why this didn't load, will repeat
* 320-01 - unclear why this didn't load, will repeat
* 323-01 - duplicate files, according to manual gating use 019-023

In addition to the above, entrie for pd209-05_t and pd294-01_t encountered errors for mappings to CCR7 and CD27:
* Alexa Fluor 405-A, CCR7
* PE-Cy7-A, CD27

In [47]:
#pd209-05_t and pd294-01_t
fcs = FCSFile(f'{root}/209-05/t_panel/PD210-12-P PDMC 2 T Cells_T1 209-05stable Tpanel PDMC_019.fcs')

In [48]:
fcs.fluoro_mappings

[{'channel': 'FSC-A', 'marker': ''},
 {'channel': 'FSC-H', 'marker': ''},
 {'channel': 'FSC-W', 'marker': ''},
 {'channel': 'SSC-A', 'marker': ''},
 {'channel': 'SSC-H', 'marker': ''},
 {'channel': 'SSC-W', 'marker': ''},
 {'channel': 'Alexa Fluor 488-A', 'marker': 'CXCR3 FITC'},
 {'channel': 'Alexa Fluor 647-A', 'marker': 'CD161 APC'},
 {'channel': 'APC-Cy7-A', 'marker': 'CD3 APCFire750'},
 {'channel': 'Alexa Fluor 405-A', 'marker': 'CD27 BV421'},
 {'channel': 'AmCyan-A', 'marker': 'CD14-19-LD'},
 {'channel': 'BV605-A', 'marker': 'Va7-2'},
 {'channel': 'BV711-A', 'marker': 'CD8'},
 {'channel': 'PE-A', 'marker': 'Vd2'},
 {'channel': 'PE-CF594-A', 'marker': 'CD45RA PE Dazz'},
 {'channel': '7-AAD-A', 'marker': 'Pan gd PECy5'},
 {'channel': 'PE-Cy5-5-A', 'marker': 'CD4'},
 {'channel': 'PE-Cy7-A', 'marker': 'CCR7 rat'},
 {'channel': 'Time', 'marker': ''}]

In [49]:
fcs = FCSFile(f'{root}/294-01/t_panel/Peri 294-01 PDMC 2 T Cells_T1_014.fcs')

In [50]:
fcs.fluoro_mappings

[{'channel': 'FSC-A', 'marker': ''},
 {'channel': 'FSC-H', 'marker': ''},
 {'channel': 'SSC-A', 'marker': ''},
 {'channel': 'SSC-W', 'marker': ''},
 {'channel': 'Alexa Fluor 488-A', 'marker': 'CXCR3 FITC'},
 {'channel': 'Alexa Fluor 647-A', 'marker': 'CD161 APC'},
 {'channel': 'APC-Cy7-A', 'marker': 'CD3 APCFire750'},
 {'channel': 'Alexa Fluor 405-A', 'marker': 'CD27 BV421'},
 {'channel': 'AmCyan-A', 'marker': 'CD14-19-LD'},
 {'channel': 'BV605-A', 'marker': 'Va7-2'},
 {'channel': 'BV711-A', 'marker': 'CD8'},
 {'channel': 'PE-A', 'marker': 'Vd2'},
 {'channel': 'PE-CF594-A', 'marker': 'CD45RA PE Dazz'},
 {'channel': '7-AAD-A', 'marker': 'Pan gd PECy5'},
 {'channel': 'PE-Cy5-5-A', 'marker': 'CD4'},
 {'channel': 'PE-Cy7-A', 'marker': 'CCR7 rat'},
 {'channel': 'Time', 'marker': ''}]

The mappings are incorrect for these two files so I cannot trust them. I will remove them from the database.

In [51]:
pdmc_t = FCSExperiment.objects(experiment_id='PDMC_T').get()

In [52]:
pdmc_t.remove_file('pd209-05_t', delete=True)

True

In [53]:
pdmc_t.remove_file('pd294-01_t', delete=True)

True

In [54]:
pdmc_t = FCSExperiment.objects(experiment_id='PDMC_T').get()
root = f'{root_drive}/Raya PD Samples/ds_friendly/PDMC'
get_fp = partial(get_fcs_file_paths, control_names=['CXCR3', 'CD25', 'CD45RA',
                                                   'CCR7'], ctrl_id='FMO')
def correct_contols(file_paths):
    primary = []
    controls = []
    for f in file_paths['primary']:
        if f.find('T2') != -1:
            controls.append(dict(control_id='CXCR3', path=f))
        if f.find('T3') != -1:
            controls.append(dict(control_id='CD27', path=f))
        if f.find('T4') != -1:
            controls.append(dict(control_id='CD45RA', path=f))
        if f.find('T5') != -1:
            controls.append(dict(control_id='CCR7', path=f))
        if f.find('T1') != -1:
            primary.append(f)
    return primary, controls

In [55]:
#pd210-14 T
filepaths = get_fp(f'{root}/210-14/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [56]:
filepaths

{'primary': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T1_034.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T2_035.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T3_036.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T4_037.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T5_038.fcs'],
 'controls': []}

In [57]:
primary, controls = correct_contols(filepaths)

In [58]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T1_034.fcs']

In [59]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T2_035.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T3_036.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T4_037.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/210-14/t_panel/Peri 210-14 PDMC T5_038.fcs'}]

In [60]:
pdmc_t.add_new_sample(sample_id=f'pd210-14_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd210-14_t and associated to PDMC_T


'5d850740975ee6db9454ab23'

In [61]:
#237-06 T
filepaths = get_fp(f'{root}/237-06/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [62]:
filepaths

{'primary': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T1_001.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T2_002.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T3_003.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T4_004.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T5_005.fcs'],
 'controls': []}

In [63]:
primary, controls = correct_contols(filepaths)

In [64]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T1_001.fcs']

In [65]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T2_002.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T3_003.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T4_004.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/237-06/t_panel/Peri 237-06 PDMC T5_005.fcs'}]

In [66]:
pdmc_t.add_new_sample(sample_id=f'pd237-06_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd237-06_t and associated to PDMC_T


'5d8507c0975ee6db9454ccde'

In [67]:
#pd264-02 T
filepaths = get_fp(f'{root}/264-02/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [68]:
primary, controls = correct_contols(filepaths)

In [69]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/264-02/t_panel/Stable 264-02 PDMC T1_026.fcs']

In [70]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/264-02/t_panel/Stable 264-02 PDMC T2_027.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/264-02/t_panel/Stable 264-02 PDMC T3_028.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/264-02/t_panel/Stable 264-02 PDMC T4_029.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/264-02/t_panel/Stable 264-02 PDMC T5_030.fcs'}]

In [71]:
pdmc_t.add_new_sample(sample_id=f'pd264-02_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd264-02_t and associated to PDMC_T


'5d8507e1975ee6db9454d5ab'

In [72]:
#pd267-02 T
filepaths = get_fp(f'{root}/267-02/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [73]:
primary, controls = correct_contols(filepaths)

In [74]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/267-02/t_panel/267-02 Peri PDMC T1_027.fcs']

In [75]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/267-02/t_panel/267-02 Peri PDMC T2_028.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/267-02/t_panel/267-02 Peri PDMC T3_029.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/267-02/t_panel/267-02 Peri PDMC T4_030.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/267-02/t_panel/267-02 Peri PDMC T5_031.fcs'}]

In [76]:
pdmc_t.add_new_sample(sample_id=f'pd267-02_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd267-02_t and associated to PDMC_T


'5d850825975ee6db9454e739'

In [77]:
#pd273-01 T
filepaths = get_fp(f'{root}/273-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [78]:
primary, controls = correct_contols(filepaths)

In [79]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/273-01/t_panel/Peri 273-01 PDMC T1_014.fcs']

In [80]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/273-01/t_panel/Peri 273-01 PDMC T2_015.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/273-01/t_panel/Peri 273-01 PDMC T3_016.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/273-01/t_panel/Peri 273-01 PDMC T4_017.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/273-01/t_panel/Peri 273-01 PDMC T5_018.fcs'}]

In [81]:
pdmc_t.add_new_sample(sample_id=f'pd273-01_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd273-01_t and associated to PDMC_T


'5d8508e8975ee6db945519d8'

In [82]:
#pd276-01 T
filepaths = get_fp(f'{root}/276-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [83]:
primary, controls = correct_contols(filepaths)

In [84]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/276-01/t_panel/Peri 276-01 PDMC T1_024.fcs']

In [85]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/276-01/t_panel/Peri 276-01 PDMC T2_025.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/276-01/t_panel/Peri 276-01 PDMC T3_026.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/276-01/t_panel/Peri 276-01 PDMC T4_027.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/276-01/t_panel/Peri 276-01 PDMC T5_028.fcs'}]

In [86]:
pdmc_t.add_new_sample(sample_id=f'pd276-01_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd276-01_t and associated to PDMC_T


'5d85095d975ee6db9455396f'

In [87]:
#pd286-03 T
filepaths = get_fp(f'{root}/286-03/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [88]:
primary, controls = correct_contols(filepaths)

In [89]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/286-03/t_panel/286-03 Peri PDMC T1_014.fcs']

In [90]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/286-03/t_panel/286-03 Peri PDMC T2_015.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/286-03/t_panel/286-03 Peri PDMC T3_016.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/286-03/t_panel/286-03 Peri PDMC T4_017.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/286-03/t_panel/286-03 Peri PDMC T5_018.fcs'}]

In [91]:
pdmc_t.add_new_sample(sample_id=f'pd286-03_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd286-03_t and associated to PDMC_T


'5d850993975ee6db945547f3'

In [92]:
#pd288-02 T
filepaths = get_fp(f'{root}/288-02/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [93]:
primary, controls = correct_contols(filepaths)

In [94]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/288-02/t_panel/Peri 288-02 Bag2 PDMC T1_006.fcs']

In [95]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/288-02/t_panel/Peri 288-02 Bag2 PDMC T2_007.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/288-02/t_panel/Peri 288-02 Bag2 PDMC T3_008.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/288-02/t_panel/Peri 288-02 Bag2 PDMC T4_009.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/288-02/t_panel/Peri 288-02 Bag2 PDMC T5_010.fcs'}]

In [96]:
pdmc_t.add_new_sample(sample_id=f'pd288-02_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd288-02_t and associated to PDMC_T


'5d850a03975ee6db9455661f'

In [97]:
#pd302-01 T
filepaths = get_fp(f'{root}/302-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [98]:
primary, controls = correct_contols(filepaths)

In [99]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/T Cell Panel_PD302-01-P PDMC T1_014.fcs',
 '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T1_044.fcs']

In [100]:
controls

[]

In [101]:
filepaths

{'primary': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/T Cell Panel_PD302-01-P PDMC T1_014.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/T Cell Panel_PD302-01-P PDMC Unstained_019.fcs',
  '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T1_044.fcs'],
 'controls': [{'control_id': 'CXCR3',
   'path': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/T Cell Panel_PD302-01-P PDMC T2 FMO CXCR3_015.fcs',
    '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T2 FMO CXCR3_045.fcs']},
  {'control_id': 'CD45RA',
   'path': ['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/T Cell Panel_PD302-01-P PDMC T4 FMO CD45RA_017.fcs',
    '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T4 FMO CD45RA_047.fcs']},
  {'control_id': 'CCR7',
   'path': ['/media/rossco/FCS_DATA/Raya PD S

In [102]:
primary = filepaths['primary'][2]

In [103]:
primary

'/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T1_044.fcs'

In [104]:
cxcr3 = [c for c in filepaths['controls'] if c['control_id'] == 'CXCR3']
cd45ra = [c for c in filepaths['controls'] if c['control_id'] == 'CD45RA']
ccr7 = [c for c in filepaths['controls'] if c['control_id'] == 'CCR7']

In [105]:
def correction(d):
    return dict(control_id=d[0]['control_id'], path=d[0]['path'][1])
cxcr3 = correction(cxcr3)
cd45ra = correction(cd45ra)
ccr7 = correction(ccr7)

In [106]:
primary

'/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T1_044.fcs'

In [107]:
controls = [cxcr3,cd45ra,ccr7]
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T2 FMO CXCR3_045.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T4 FMO CD45RA_047.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/302-01/t_panel/Peri 302-01 PDMC T3 FMO CCR7_046.fcs'}]

In [108]:
pdmc_t.add_new_sample(sample_id=f'pd302-01_t', 
                      controls=controls, 
                      file_path=primary)

Generating main file entry...
Generating file entries for controls...
Successfully created pd302-01_t and associated to PDMC_T


'5d850a13975ee6db94556a3a'

In [109]:
#pd305-02 T
filepaths = get_fp(f'{root}/305-02/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [110]:
primary, controls = correct_contols(filepaths)

In [111]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/305-02/t_panel/305-02 Stable PDMC T1_030.fcs']

In [112]:
controls

[{'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/305-02/t_panel/305-02 Stable PDMC T5 CD27_034.fcs'}]

In [113]:
pdmc_t.add_new_sample(sample_id=f'pd305-02_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd305-02_t and associated to PDMC_T


'5d850a21975ee6db94556e26'

In [114]:
#pd308-03R T
filepaths = get_fp(f'{root}/308-03R/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [115]:
primary, controls = correct_contols(filepaths)

In [116]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/308-03R/t_panel/308-03R Peri PDMC T1_032.fcs']

In [117]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/308-03R/t_panel/308-03R Peri PDMC T2_033.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/308-03R/t_panel/308-03R Peri PDMC T3_034.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/308-03R/t_panel/308-03R Peri PDMC T4_036.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/308-03R/t_panel/308-03R Peri PDMC T5_037.fcs'}]

In [118]:
pdmc_t.add_new_sample(sample_id=f'pd308-03R_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd308-03R_t and associated to PDMC_T


'5d850a66975ee6db945580c4'

In [119]:
#pd315-01 T
filepaths = get_fp(f'{root}/315-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [120]:
primary, controls = correct_contols(filepaths)

In [121]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-01/t_panel/315-01 Peri PDMC T1_015.fcs']

In [122]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-01/t_panel/315-01 Peri PDMC T2_016.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-01/t_panel/315-01 Peri PDMC T3_017.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-01/t_panel/315-01 Peri PDMC T4_018.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-01/t_panel/315-01 Peri PDMC T5_019.fcs'}]

In [123]:
pdmc_t.add_new_sample(sample_id=f'pd315-01_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd315-01_t and associated to PDMC_T


'5d850a86975ee6db94558986'

In [124]:
#pd315-02 T
filepaths = get_fp(f'{root}/315-02/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [125]:
primary, controls = correct_contols(filepaths)

In [126]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-02/t_panel/315-02 Stable PDMC T1_020.fcs']

In [127]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-02/t_panel/315-02 Stable PDMC T2_021.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-02/t_panel/315-02 Stable PDMC T3_022.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-02/t_panel/315-02 Stable PDMC T4_023.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/315-02/t_panel/315-02 Stable PDMC T5_024.fcs'}]

In [128]:
pdmc_t.add_new_sample(sample_id=f'pd315-02_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd315-02_t and associated to PDMC_T


'5d850ac2975ee6db945599e4'

In [129]:
#pd316-01 T
filepaths = get_fp(f'{root}/316-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [130]:
primary, controls = correct_contols(filepaths)

In [131]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/316-01/t_panel/316-01 Peri  PDMC T1_021.fcs']

In [132]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/316-01/t_panel/316-01 Peri  PDMC T2 CXCR3_022.fcs'}]

In [133]:
pdmc_t.add_new_sample(sample_id=f'pd316-01_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd316-01_t and associated to PDMC_T


'5d850b28975ee6db9455b4c6'

In [134]:
#pd320-01 T
filepaths = get_fp(f'{root}/320-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [135]:
primary, controls = correct_contols(filepaths)

In [136]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/320-01/t_panel/320-01 Peri PDMC T1_015.fcs']

In [137]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/320-01/t_panel/320-01 Peri PDMC T2_016.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/320-01/t_panel/320-01 Peri PDMC T3_017.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/320-01/t_panel/320-01 Peri PDMC T4_018.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/320-01/t_panel/320-01 Peri PDMC T5_019.fcs'}]

In [138]:
pdmc_t.add_new_sample(sample_id=f'pd320-01_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd320-01_t and associated to PDMC_T


'5d850b9a975ee6db9455d421'

In [139]:
#pd323-01 T
filepaths = get_fp(f'{root}/323-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [140]:
primary, controls = correct_contols(filepaths)

In [141]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/323-01/t_panel/323-01 Stable PDMC T1_019.fcs',
 '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/323-01/t_panel/T Cell Panel_323-01 PDMC T1_014.fcs']

In [142]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/323-01/t_panel/323-01 Stable PDMC T2_020.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/323-01/t_panel/323-01 Stable PDMC T3_021.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/323-01/t_panel/323-01 Stable PDMC T4_022.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/323-01/t_panel/323-01 Stable PDMC T5_023.fcs'}]

In [143]:
pdmc_t.add_new_sample(sample_id=f'pd323-01_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd323-01_t and associated to PDMC_T


'5d850beb975ee6db9455ea15'

In [144]:
#pd318-01 T
filepaths = get_fp(f'{root}/318-01/t_panel')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


In [145]:
primary, controls = correct_contols(filepaths)

In [146]:
primary

['/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/318-01/t_panel/318-01 Stable PBMC T1_015.fcs']

In [147]:
controls

[{'control_id': 'CXCR3',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/318-01/t_panel/318-01 Stable PBMC T2_016.fcs'},
 {'control_id': 'CD27',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/318-01/t_panel/318-01 Stable PBMC T3_017.fcs'},
 {'control_id': 'CD45RA',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/318-01/t_panel/318-01 Stable PBMC T4_018.fcs'},
 {'control_id': 'CCR7',
  'path': '/media/rossco/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/318-01/t_panel/318-01 Stable PBMC T5_019.fcs'}]

In [148]:
pdmc_t.add_new_sample(sample_id=f'pd318-01_t', 
                      controls=controls, 
                      file_path=primary[0])

Generating main file entry...
Generating file entries for controls...
Successfully created pd318-01_t and associated to PDMC_T


'5d850c3a975ee6db9455ff70'